In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
df=pd.read_csv("C:\\Users\\varun\\OneDrive\\Desktop\\kaggle\\dataset\\Regression\\insurance.csv")

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
num_col=df.select_dtypes(include=['int64','float64']).columns
for col in num_col:
    print(f"{col}   {df[col].skew()}")


In [ ]:
df['charges_log']=np.log(df['charges']+1)
# df['charges_log'].skew()
sns.histplot(df['charges_log'],kde=True)

In [ ]:
sns.histplot(df['charges'],kde=True)


In [ ]:
# df.drop('charges',axis=1,inplace=True)
for col in num_col:
    plt.figure(figsize=(6,4))
    sns.boxplot(df[col])

In [ ]:
q1=df['bmi'].quantile(0.25)
q3=df['bmi'].quantile(0.75)
IQR=q3-q1
lower_limit,upper_limit=(q1-1.5*IQR),(q3+1.5*IQR)
df['bmi']=df['bmi'].apply(lambda x: upper_limit 
                          if x> upper_limit else (lower_limit if  x < lower_limit else x ))

In [ ]:
# df['bmi']
sns.boxplot(df['bmi'])

In [ ]:
df.info()

In [ ]:
char_col=df.select_dtypes(include='object').columns
for col in char_col:
    print(f"{col}       {df[col].unique()}")

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder ,StandardScaler,LabelEncoder
preprocessor=ColumnTransformer([
    ('num',StandardScaler(),num_col),
    ('cat',OneHotEncoder(),char_col)
    
])
x_transformed=preprocessor.fit_transform(df.drop('charges_log',axis=1))
# x_pre=preprocessor.drop('charges_log',axis=1)
# x_transfor=pd.DataFrame(preprocessor)

In [ ]:
# x_transformed



In [ ]:
# char_col=df.select_dtypes(exclude='object').columns
# for col in char_col:
x,y=df.drop('charges_log',axis=1),df['charges_log']
from sklearn.preprocessing import OneHotEncoder ,StandardScaler,LabelEncoder
from sklearn.pipeline import Pipeline
simple_pipe=Pipeline([
    ('ohe',OneHotEncoder(drop='first',sparse_output=False)),
    ('ss',StandardScaler()),
    ('le',LabelEncoder())
])
df
df['sex']=simple_pipe.named_steps['le'].fit_transform(df['sex'])
df['smoker']=simple_pipe.named_steps['le'].fit_transform(df['smoker'])
encoder=simple_pipe.named_steps['ohe'].fit_transform(df[['region']])
encoder_final=pd.DataFrame(encoder,columns=simple_pipe.named_steps['ohe'].get_feature_names_out())
df=pd.concat([df,encoder_final],axis=1)


# df.drop('region',axis=1,inplace=True)

    

In [ ]:
x,y=df.drop('charges_log',axis=1),df['charges_log']
df.info()

# df.drop('region',axis=1,inplace=True)

In [ ]:
from sklearn.linear_model import LinearRegression,ElasticNet,Lasso,Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler
model_pipe=Pipeline([
    ('poly',PolynomialFeatures(degree=3)),
    ('lg',LinearRegression()),
    ('rd',Ridge()),
    ('ls',Lasso()),
    ('elastic',ElasticNet())
])
pram_grid={'alpha':[0.01,0.1,1,10,100]}
grid_search=GridSearchCV(ElasticNet(),pram_grid,cv=5,scoring='r2')



In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)
x_ss_train=simple_pipe.named_steps['ss'].fit_transform(x_train)
x_ss_test=simple_pipe.named_steps['ss'].transform(x_test)
model_pipe.named_steps['lg'].fit(x_ss_train,y_train)
y_pred=model_pipe.named_steps['lg'].predict(x_ss_test)
r2_linear=r2_score(y_test,y_pred)
r2_linear
# x.info()

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_transformed,y,test_size=0.3,random_state=42)
x_ss_train=simple_pipe.named_steps['ss'].fit_transform(x_train)
x_ss_test=simple_pipe.named_steps['ss'].transform(x_test)
model_pipe.named_steps['lg'].fit(x_ss_train,y_train)
y_pred=model_pipe.named_steps['lg'].predict(x_ss_test)
r2_linear=r2_score(y_test,y_pred)
r2_linear
# x.info()

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)
x_poly_train=model_pipe.named_steps['poly'].fit_transform(x_train)
x_poly_test=model_pipe.named_steps['poly'].transform(x_test)
model_pipe.named_steps['lg'].fit(x_poly_train,y_train)
y_pred=model_pipe.named_steps['lg'].predict(x_poly_test)
r2_poly=r2_score(y_test,y_pred)
r2_poly

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_transformed,y,test_size=0.3,random_state=42)
x_poly_train=model_pipe.named_steps['poly'].fit_transform(x_train)
x_poly_test=model_pipe.named_steps['poly'].transform(x_test)
model_pipe.named_steps['lg'].fit(x_poly_train,y_train)
y_pred=model_pipe.named_steps['lg'].predict(x_poly_test)
r2_poly=r2_score(y_test,y_pred)
r2_poly

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)
x_ss_train=simple_pipe.named_steps['ss'].fit_transform(x_train)
x_ss_test=simple_pipe.named_steps['ss'].transform(x_test)
grid_search=GridSearchCV(Ridge(alpha=1.0),pram_grid,cv=5,scoring='r2')
model_pipe.named_steps['rd'].fit(x_train,y_train)
# best_elimate=grid_search.best_estimator_
y_pred=model_pipe.named_steps['rd'].predict(x_test)
r2_ridge=r2_score(y_test,y_pred)
r2_ridge

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_transformed,y,test_size=0.3,random_state=42)
x_ss_train=simple_pipe.named_steps['ss'].fit_transform(x_train)
x_ss_test=simple_pipe.named_steps['ss'].transform(x_test)
grid_search=GridSearchCV(Ridge(alpha=1.0),pram_grid,cv=5,scoring='r2')
model_pipe.named_steps['rd'].fit(x_ss_train,y_train)
y_pred=model_pipe['rd'].predict(x_ss_test)
r2_ridge=r2_score(y_test,y_pred)
r2_ridge

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)
x_ss_train=simple_pipe.named_steps['ss'].fit_transform(x_train)
x_ss_test=simple_pipe.named_steps['ss'].transform(x_test)
grid_search=GridSearchCV(Lasso(alpha=0.01),pram_grid,cv=5,scoring='r2')
model_pipe.named_steps['ls'].fit(x_train,y_train)
# best_elimate=grid_search.best_estimator_
y_pred=model_pipe.named_steps['ls'].predict(x_test)
r2_lasso=r2_score(y_test,y_pred)
# r2_lasso
# model_pipe.named_steps['ls'].coef_
grid_search.best_params_

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_transformed,y,test_size=0.3,random_state=42)
x_ss_train=simple_pipe.named_steps['ss'].fit_transform(x_train)
x_ss_test=simple_pipe.named_steps['ss'].transform(x_test)
grid_search=GridSearchCV(Lasso(alpha=0.01),pram_grid,cv=5,scoring='r2')
model_pipe.named_steps['ls'].fit(x_ss_train,y_train)
# best_elimate=grid_search.best_estimator_
y_pred=model_pipe.named_steps['ls'].predict(x_ss_test)
r2_lasso=r2_score(y_test,y_pred)
r2_lasso

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)
x_ss_train=simple_pipe.named_steps['ss'].fit_transform(x_train)
x_ss_test=simple_pipe.named_steps['ss'].transform(x_test)
grid_search=GridSearchCV(ElasticNet(),pram_grid,cv=5,scoring='r2')
model_pipe.named_steps['elastic'].fit(x_train,y_train)
# best_elimate=grid_search.best_estimator_
y_pred=model_pipe.named_steps['elastic'].predict(x_test)
r2_lasso=r2_score(y_test,y_pred)
r2_lasso

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_transformed,y,test_size=0.3,random_state=42)
x_ss_train=simple_pipe.named_steps['ss'].fit_transform(x_train)
x_ss_test=simple_pipe.named_steps['ss'].transform(x_test)
grid_search=GridSearchCV(ElasticNet(),pram_grid,cv=5,scoring='r2')
model_pipe.named_steps['elastic'].fit(x_ss_train,y_train)
# best_elimate=grid_search.best_estimator_
y_pred=model_pipe.named_steps['elastic'].predict(x_ss_test)
r2_elastic=r2_score(y_test,y_pred)
r2_elastic